#### Configurations

In [1]:
GITLAB_URL='https://gitlab.com'
GITLAB_ACCESS_TOKEN=''
FILTER_DATE_BEGIN='2020-06-01'
FILTER_DATE_END='2020-06-31'
FILTER_MILESTONE='Bem1.7'
FILTER_SEARCH='Bem'
FILTER_AUTHOR=None
FILTER_ONLY_MEMBER=True
DEBUG=True

In [2]:
SERVICE_COST_PER_HOUR=50
PERSON_COST_PER_HOUR = {
    "flaviostutz": 50
}

### Fetch data from Gitlab

In [3]:
import pandas as pd
import gitlab
import modules.gitlabspent as gs
gl = gitlab.Gitlab(GITLAB_URL, private_token=GITLAB_ACCESS_TOKEN)

In [4]:
if GITLAB_ACCESS_TOKEN is None or GITLAB_ACCESS_TOKEN == '':
  raise 'GITLAB_ACCESS_TOKEN is required'
entries = gs.fetch_time_entries(gl, filter_by_author=FILTER_AUTHOR, filter_by_date_begin=FILTER_DATE_BEGIN, filter_by_date_end=FILTER_DATE_END, filter_by_project_membership=FILTER_ONLY_MEMBER, filter_by_search=FILTER_SEARCH, filter_by_milestone=FILTER_MILESTONE, debug=DEBUG)
print('%s time entries fetched' %(len(entries)))

Getting projects. search=Bem only_member=True

Getting issues for project=Bem Promotions 
Getting issues for project=Bem MySQL 
Getting issues for project=Bem Frontend . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Getting issues for project=Bem Backend . 
Getting issues for project=Bem estar Saúde . 2 time entries fetched


In [5]:
df = pd.DataFrame()
for e in entries[0]:
  df = df.append(e, ignore_index=True)

## Raw spend table

In [6]:
df = df.sort_values(['date','note_author'])
df2 = df.astype({"duration": int, "issue_iid": int})
# df2['duration'] = df2['duration'].map(gs.format_duration)
df2s = df2.style.set_properties(**{'text-align': 'left'}, subset=pd.IndexSlice[:, ['issue_title']])
display(df2s)

,date,duration,issue_iid,issue_title,milestone_title,note_author
7,2020-06-02,2100,84,Permitir cadastramento com mesmo email e CPF caso usuário tenha perdido o tempo de validade do link de ativação da conta,Bem1.7,danismerim
5,2020-06-05,1200,88,"""página não encontrada""",Bem1.7,danismerim
0,2020-06-05,3600,93,Excluir do mecanismo de busca perfis com hasPaid=false,Bem1.7,flaviostutz
2,2020-06-05,14400,91,Instalar kibana - pra não ficar público o perfil profissional ainda não aprovado,Bem1.7,flaviostutz
1,2020-06-06,1200,92,A busca de profissionais não é atualizada depois do profissional atualizar seus dados de perfil,Bem1.7,danismerim
6,2020-06-06,300,86,Perfil profissional - meus dados - edição não salva ao clicar no botão,Bem1.7,danismerim
3,2020-06-11,6000,90,Não permitir novo perfil profissional público antes de aprovação,Bem1.7,danismerim
4,2020-06-12,3600,89,"Notificação refere-se a todos como ""médico"" (e no masculino)",Bem1.7,wedla1
8,2020-06-12,900,37,"alterar textos: ""Confirmação"" para ""validação""",Bem1.7,wedla1
9,2020-06-12,900,28,"Profissional - Ao escrever relatório - tirar obrigatoriedade do campo ""observações""",Bem1.7,wedla1


# Service cost by issue

In [7]:
from IPython.display import display, Markdown
if FILTER_MILESTONE!=None:    
  df3 = df2.groupby(['issue_iid','issue_title'])
else:
  df3 = df2.groupby(['milestone_title', 'issue_iid','issue_title'])    
df4 = df3['duration'].sum()
df5 = pd.DataFrame(df4)
df5 = df5.sort_values(['duration','issue_iid'], ascending=False)
df5['cost'] = df5['duration'] * (SERVICE_COST_PER_HOUR/3600)

df6 = df5.copy()
df6['cost'] = df6['cost'].apply(lambda x: '%.2f' %x)
df6 = df6.reset_index()
print('')
if FILTER_MILESTONE!=None:
  display(Markdown('## Milestone %s' %FILTER_MILESTONE))  
display(Markdown('### Period: %s to %s' %(FILTER_DATE_BEGIN, FILTER_DATE_END)))
display(Markdown('Price: %.2f/h' %(SERVICE_COST_PER_HOUR)))
df6['duration'] = df6['duration'].apply(gs.format_duration)
df6s = df6.style.set_properties(**{'text-align': 'left'}, subset=pd.IndexSlice[:, ['issue_title']])
display(df6s)

display(Markdown('### Total cost:     %.2f' %df5['cost'].sum()))
display(Markdown('Total duration: %s' %gs.format_duration(df5['duration'].sum())))
print('')

## Milestone Bem1.7

### Period: 2020-06-01 to 2020-06-31

Price: 50.00/h

,issue_iid,issue_title,duration,cost
0,91,Instalar kibana - pra não ficar público o perfil profissional ainda não aprovado,4h 0m,200.00
1,90,Não permitir novo perfil profissional público antes de aprovação,1h 40m,83.33
2,93,Excluir do mecanismo de busca perfis com hasPaid=false,1h 0m,50.00
3,89,"Notificação refere-se a todos como ""médico"" (e no masculino)",1h 0m,50.00
4,84,Permitir cadastramento com mesmo email e CPF caso usuário tenha perdido o tempo de validade do link de ativação da conta,0h 35m,29.17
5,92,A busca de profissionais não é atualizada depois do profissional atualizar seus dados de perfil,0h 20m,16.67
6,88,"""página não encontrada""",0h 20m,16.67
7,37,"alterar textos: ""Confirmação"" para ""validação""",0h 15m,12.50
8,28,"Profissional - Ao escrever relatório - tirar obrigatoriedade do campo ""observações""",0h 15m,12.50
9,86,Perfil profissional - meus dados - edição não salva ao clicar no botão,0h 5m,4.17


### Total cost:     475.00

Total duration: 9h 30m

# Total spent by person

In [8]:
df3 = df2.groupby(['note_author'])
df4 = df3['duration'].sum()
df5 = pd.DataFrame(df4)
df5 = df5.sort_values(['duration'], ascending=False)
df5 = df5.reset_index()
# df5['cost'] = df5['duration'] * (PERSON_COST_PER_HOUR.get(df5['note_author'].str)/3600)
# df5['cost'] = df5['cost'].apply(lambda x: '%.2f' %x)
df6 = df5.copy()
df6['duration'] = df6['duration'].apply(gs.format_duration)
display(df6)

,note_author,duration
0,flaviostutz,5h 0m
1,danismerim,3h 0m
2,wedla1,1h 30m


In [9]:
for v in df5['note_author']:
  print('')
  display(Markdown('### @%s' %(v)))
  if not v in PERSON_COST_PER_HOUR:
    print('No cost per hour set')
    continue
  display(Markdown('Period: %s to %s' %(FILTER_DATE_BEGIN, FILTER_DATE_END)))
  cost_per_hour = PERSON_COST_PER_HOUR.get(v)
  display(Markdown('Cost: %.2f/h' %(cost_per_hour)))

  df3 = df2[df2['note_author'].str.contains(v)]
  df3 = df3.drop(['note_author', 'date'], axis=1)
  df3g = df3.groupby(['issue_iid','issue_title','milestone_title'])
  df6 = df3g['duration'].sum()
  df6 = pd.DataFrame(df6)
    
  df6 = df6.sort_values(['duration','issue_iid'], ascending=False)
  df6['cost'] = df6['duration'] * (cost_per_hour/3600)
    
  df6 = df6.copy()
  total_cost = df6['cost'].sum()
  df6['cost'] = df6['cost'].apply(lambda x: '%.2f' %x)
  df6['duration'] = df6['duration'].apply(gs.format_duration)
  df6 = df6.reset_index()

  df6s = df6.style.set_properties(**{'text-align': 'left'}, subset=pd.IndexSlice[:, ['issue_title']])
  display(df6s)
    
  display(Markdown('#### Total: %.2f' %(total_cost)))
  print('')

### @flaviostutz

Period: 2020-06-01 to 2020-06-31

Cost: 50.00/h

,issue_iid,issue_title,milestone_title,duration,cost
0,91,Instalar kibana - pra não ficar público o perfil profissional ainda não aprovado,Bem1.7,4h 0m,200.00
1,93,Excluir do mecanismo de busca perfis com hasPaid=false,Bem1.7,1h 0m,50.00


#### Total: 250.00

### @danismerim

No cost per hour set



### @wedla1

No cost per hour set
